In [3]:
import pandas as pd
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [4]:
options = Options()
options.add_argument("--disable-notifications")
options.add_argument("--disable-popup-blocking")
driver = webdriver.Chrome(options=options)

In [5]:
league_url = 'https://fbref.com/en/comps/9/Premier-League-Stats'

#years = list(range(2024, 2022, -1))
years = [2024]

In [8]:
def scrapePlayersFBREF(league, NumTeams, years):

    keepers=[]
    defenders=[]
    midfielders=[]
    attackers=[]
    all_positions = [keepers, defenders, midfielders, attackers]

    data = requests.get(league)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0] #css selector to select table with class name stats_table

    links = [l.get('href') for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]  #only want the links with /squads/ in it
    team_urls = [f"https://fbref.com{l}" for l in links] #create list of all the teams urls/links

    for team in team_urls[0:NumTeams]:
        data = requests.get(team)
        soup = BeautifulSoup(data.text)
        print(team)
        
        for year in years:
            print(year)
            driver.get(team)    
            time.sleep(2)

            
            try:
                driver.find_element(By.CSS_SELECTOR, 'button[mode=primary]').click()
            except Exception:
                pass

            previous_season = soup.select("div.prevnext a")[0].get("href")
            team = f"https://fbref.com/{previous_season}"

            for ind in range(len(all_positions)):

                if ind == 0:
                    value = ["keeper_adv"]
                elif ind == 1:
                    value = ["defense"]
                elif ind == 2:
                    value = ["passing", "possession"]
                elif ind == 3:
                    value = ["shooting", "gca"]
                else:
                    Exception
                time.sleep(2)
                
                d={}

                for stat in value:
                    togglep90 =  WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, f"button[id*='stats_{stat}_']")))
                    tablename = driver.find_element(By.ID, f"all_stats_{stat}")
                    driver.execute_script("arguments[0].scrollIntoView();", tablename)

                    time.sleep(2)
                    togglep90.click()

                    page_source = BeautifulSoup(driver.page_source, 'html.parser')
                    tables = page_source.findAll('table', id=lambda x: x and x.startswith(f'stats_{stat}'))[0]
                    d[stat] = pd.read_html(str(tables))[0]

                d[stat]["Season"] = year

                if len(d) > 1:
                    all_positions[ind].append(d[value[0]].merge(d[value[1]]))
                else:
                    all_positions[ind].append(d[value[0]])
                

    driver.quit()

    GKdf = pd.concat(keepers)
    DEFdf= pd.concat(defenders)
    MIDdf = pd.concat(midfielders)
    ATTdf = pd.concat(attackers)

    all_df = [GKdf, DEFdf, MIDdf, ATTdf]

    for df in all_df:
        df.columns = ['_'.join(col).strip() for col in df.columns.values]

    all_df = [GKdf, DEFdf, MIDdf, ATTdf]

    ATTdf = ATTdf.reset_index()
    MIDdf = MIDdf.reset_index()
    DEFdf = DEFdf.reset_index()
    GKdf = GKdf.reset_index()

    ATTdf = ATTdf.drop('index', axis=1)
    MIDdf = MIDdf.drop('index', axis=1)
    DEFdf = DEFdf.drop('index', axis=1)
    GKdf = GKdf.drop('index', axis=1)

    ATTdf = ATTdf.set_index('Unnamed: 0_level_0_Player')
    MIDdf = MIDdf.set_index('Unnamed: 0_level_0_Player')
    DEFdf = DEFdf.set_index('Unnamed: 0_level_0_Player')
    GKdf = GKdf.set_index('Unnamed: 0_level_0_Player')

    ATTdf = ATTdf.drop(["Squad Total", "Opponent Total"])
    MIDdf = MIDdf.drop(["Squad Total", "Opponent Total"])
    DEFdf = DEFdf.drop(["Squad Total", "Opponent Total"])
    GKdf = GKdf.drop(["Squad Total", "Opponent Total"])

    ATTdf = ATTdf.reset_index()
    MIDdf = MIDdf.reset_index()
    DEFdf = DEFdf.reset_index()
    GKdf = GKdf.reset_index()

    ATTdf = ATTdf[ATTdf["Unnamed: 2_level_0_Pos"].str.contains("FW")]
    MIDdf = MIDdf[MIDdf["Unnamed: 2_level_0_Pos"].str.contains("MF")]
    DEFdf = DEFdf[DEFdf["Unnamed: 2_level_0_Pos"].str.contains("DF")]

    # remove players with few games played (which can often skew graphs/act as outliers)
    ATTdf = ATTdf[ATTdf['Unnamed: 4_level_0_90s'] >= 3.0]
    MIDdf = MIDdf[MIDdf['Unnamed: 4_level_0_90s'] >= 3.0]
    DEFdf = DEFdf[DEFdf['Unnamed: 4_level_0_90s'] >= 3.0]
    GKdf = GKdf[GKdf['Unnamed: 4_level_0_90s'] >= 3.0]

    ATTdf = ATTdf.reset_index()
    MIDdf = MIDdf.reset_index()
    DEFdf = DEFdf.reset_index()

    ATTdf = ATTdf.drop("index", axis=1)
    MIDdf = MIDdf.drop("index", axis=1)
    DEFdf = DEFdf.drop("index", axis=1)

    ATTdf = ATTdf.drop(["Unnamed: 1_level_0_Nation", "Unnamed: 3_level_0_Age", "Unnamed: 4_level_0_90s", "Standard_SoT", "Standard_SoT/90", 
                        "Standard_G/SoT", "Standard_FK", "Standard_PK", "Standard_PKatt", "Expected_npxG/Sh", "Expected_G-xG", "SCA_SCA", 
                        "SCA Types_PassLive", "SCA Types_PassDead", "SCA Types_Sh", "SCA Types_Fld", "SCA Types_Def",
                        "GCA_GCA",  "GCA Types_PassLive", "GCA Types_PassDead", "GCA Types_TO", "GCA Types_Sh", "GCA Types_Fld", "GCA Types_Def",
                        "Unnamed: 21_level_0_Matches", "Unnamed: 22_level_0_Matches"], axis=1)
    
    MIDdf = MIDdf.drop(["Unnamed: 1_level_0_Nation", "Unnamed: 3_level_0_Age", "Unnamed: 4_level_0_90s", "Total_TotDist", "Short_Cmp", "Short_Att",
                        "Short_Cmp%", "Medium_Cmp", "Medium_Att", "Medium_Cmp%", "Long_Cmp",
                        "Long_Att", "Long_Cmp%", "Unnamed: 20_level_0_xAG", "Expected_A-xAG", "Unnamed: 26_level_0_CrsPA", 
                        "Touches_Def Pen", "Touches_Def 3rd", "Touches_Mid 3rd", "Touches_Att 3rd", "Touches_Att Pen", "Touches_Live",
                        "Take-Ons_Succ", "Take-Ons_Tkld", "Take-Ons_Tkld%", "Carries_Carries", "Carries_TotDist", "Carries_PrgDist",
                        "Carries_Mis", "Carries_Dis", "Unnamed: 28_level_0_Matches", "Receiving_Rec", "Receiving_PrgR", "Unnamed: 27_level_0_Matches"],
                        axis=1)
    
    DEFdf = DEFdf.drop(["Unnamed: 1_level_0_Nation", "Unnamed: 3_level_0_Age", "Unnamed: 4_level_0_90s", "Tackles_Att 3rd", "Challenges_Tkl", "Challenges_Att",
                        "Challenges_Tkl%", "Challenges_Lost", "Blocks_Pass", "Unnamed: 21_level_0_Matches"], axis=1)
    
    GKdf = GKdf.drop(["Unnamed: 1_level_0_Nation", "Unnamed: 3_level_0_Age", "Unnamed: 4_level_0_90s", "Goals_FK", "Goals_OG", "Goal Kicks_Att",
                    "Crosses_Opp", "Crosses_Stp", "Expected_/90", "Unnamed: 30_level_0_Matches"], axis=1)

    ATTdf.columns = ['Player', 'Pos', 'Goals', 'Shots','SOT%', 'Sh/90', 'G/Sh', 'Dist', 'xG', 'npxG', 'np:g-xG', 
    'SCA/90', 'SCA(TakeOn)', 'GCA/90', 'Season']

    MIDdf.columns = ['Player', 'Pos', 'PassCmp', 'PassAtt', 'PassComp%',
    'PassPrgDist', 'Assists', 'xA', 'KP', 'IntoFin1/3', 'PPA', 'ProgPass', 'Touches', 'TakeOnsAtt', 'TakeOnsSuc%',
    'PrgCar', 'Carrs1/3', 'CPA', 'Season']

    DEFdf.columns = ['Player', 'Pos', 'Tackles', 'TacklesWon', 'TacklesDef1/3', 'TacklesMid1/3',
    'Blocks', 'ShotsBlocked', 'Interceptions', 'Tkl+Int', 'Clearances', 'ErrorsToShot', 'Season']

    GKdf.columns = ['Player', 'Pos', 'GA', 'PKGA', 'CKGA' , 'PSxG', 'PSxG/SoT',
    'PSxG+/-GA', 'LaunchCmp', 'LaunchAtt', 'LaunchCmp%', 'PassesAtt', 'Throws', 'Launch%', 'AvgPassLen', 'GKLaunch%',
    'AvgGKLen', 'CrossesStop%', 'OPA', 'OPA/90', 'AvgDistActions', 'Season']

    return GKdf, DEFdf, MIDdf, ATTdf

In [9]:
GKs, DEFs, MIDs, ATTs = scrapePlayersFBREF(league_url, 5, years)

https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats
2024


C:\Users\tomas.hehir\AppData\Local\Temp\ipykernel_36128\3221093467.py:62: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  d[stat] = pd.read_html(str(tables))[0]
C:\Users\tomas.hehir\AppData\Local\Temp\ipykernel_36128\3221093467.py:62: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  d[stat] = pd.read_html(str(tables))[0]
C:\Users\tomas.hehir\AppData\Local\Temp\ipykernel_36128\3221093467.py:62: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  d[stat] = pd.read_html(str(tables))[0]
C:\Users\tomas.hehir\AppData\Local\Temp\ipykernel_36128\3221093467.py:62: FutureWarning: Passing literal html to 'read_html' is deprecated and will 

https://fbref.com/en/squads/18bb7c10/Arsenal-Stats
2024


C:\Users\tomas.hehir\AppData\Local\Temp\ipykernel_36128\3221093467.py:62: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  d[stat] = pd.read_html(str(tables))[0]
C:\Users\tomas.hehir\AppData\Local\Temp\ipykernel_36128\3221093467.py:62: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  d[stat] = pd.read_html(str(tables))[0]
C:\Users\tomas.hehir\AppData\Local\Temp\ipykernel_36128\3221093467.py:62: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  d[stat] = pd.read_html(str(tables))[0]
C:\Users\tomas.hehir\AppData\Local\Temp\ipykernel_36128\3221093467.py:62: FutureWarning: Passing literal html to 'read_html' is deprecated and will 

https://fbref.com/en/squads/822bd0ba/Liverpool-Stats
2024


C:\Users\tomas.hehir\AppData\Local\Temp\ipykernel_36128\3221093467.py:62: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  d[stat] = pd.read_html(str(tables))[0]
C:\Users\tomas.hehir\AppData\Local\Temp\ipykernel_36128\3221093467.py:62: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  d[stat] = pd.read_html(str(tables))[0]
C:\Users\tomas.hehir\AppData\Local\Temp\ipykernel_36128\3221093467.py:62: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  d[stat] = pd.read_html(str(tables))[0]
C:\Users\tomas.hehir\AppData\Local\Temp\ipykernel_36128\3221093467.py:62: FutureWarning: Passing literal html to 'read_html' is deprecated and will 

https://fbref.com/en/squads/8602292d/Aston-Villa-Stats
2024


C:\Users\tomas.hehir\AppData\Local\Temp\ipykernel_36128\3221093467.py:62: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  d[stat] = pd.read_html(str(tables))[0]
C:\Users\tomas.hehir\AppData\Local\Temp\ipykernel_36128\3221093467.py:62: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  d[stat] = pd.read_html(str(tables))[0]
C:\Users\tomas.hehir\AppData\Local\Temp\ipykernel_36128\3221093467.py:62: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  d[stat] = pd.read_html(str(tables))[0]
C:\Users\tomas.hehir\AppData\Local\Temp\ipykernel_36128\3221093467.py:62: FutureWarning: Passing literal html to 'read_html' is deprecated and will 

https://fbref.com/en/squads/361ca564/Tottenham-Hotspur-Stats
2024


C:\Users\tomas.hehir\AppData\Local\Temp\ipykernel_36128\3221093467.py:62: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  d[stat] = pd.read_html(str(tables))[0]
C:\Users\tomas.hehir\AppData\Local\Temp\ipykernel_36128\3221093467.py:62: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  d[stat] = pd.read_html(str(tables))[0]
C:\Users\tomas.hehir\AppData\Local\Temp\ipykernel_36128\3221093467.py:62: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  d[stat] = pd.read_html(str(tables))[0]
C:\Users\tomas.hehir\AppData\Local\Temp\ipykernel_36128\3221093467.py:62: FutureWarning: Passing literal html to 'read_html' is deprecated and will 

In [10]:
GKs.to_csv("C:/Users/tomas.hehir/Source/Repos/Footy/data/PL/TOP5Keepers.csv")
DEFs.to_csv("C:/Users/tomas.hehir/Source/Repos/Footy/data/PL/TOP5Defenders.csv")
MIDs.to_csv("C:/Users/tomas.hehir/Source/Repos/Footy/data/PL/TOP5Midfielders.csv")
ATTs.to_csv("C:/Users/tomas.hehir/Source/Repos/Footy/data/PL/TOP5Attackers.csv")